In [41]:
import pandas as pd

# Load the datasets
athelet_df = pd.read_csv('data/athlete_events.csv')
gdp_df = pd.read_csv('data/gdp_data.csv', skiprows=4)


## Cleaning datasets

# Transform GDP data using melt
gdp_data_long = gdp_df.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                             var_name='Year', value_name='GDP')
gdp_data_long = gdp_data_long[gdp_data_long['Year'] != 'Unnamed: 67']
gdp_df = gdp_data_long
# convert year values into an integer type
gdp_data_long['Year'] = gdp_data_long['Year'].astype(int)

# Rename columns
gdp_df = gdp_df.rename(columns={
    'Country Name': 'CountryName',
    'Country Code': 'CountryCode',
    'Indicator Name': 'IndicatorName',
    'Indicator Code': 'IndicatorCode',
})

athelet_df = athelet_df.rename(columns={
    'Team': 'CountryName'
})

# Check the each dataset
print(gdp_df.head())
print(athelet_df.head())


                   CountryName CountryCode      IndicatorName   IndicatorCode  \
0                        Aruba         ABW  GDP (current US$)  NY.GDP.MKTP.CD   
1  Africa Eastern and Southern         AFE  GDP (current US$)  NY.GDP.MKTP.CD   
2                  Afghanistan         AFG  GDP (current US$)  NY.GDP.MKTP.CD   
3   Africa Western and Central         AFW  GDP (current US$)  NY.GDP.MKTP.CD   
4                       Angola         AGO  GDP (current US$)  NY.GDP.MKTP.CD   

   Year           GDP  
0  1960           NaN  
1  1960  2.112502e+10  
2  1960  5.377778e+08  
3  1960  1.044764e+10  
4  1960           NaN  
   ID                      Name Sex   Age  Height  Weight     CountryName  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  De

In [45]:
## Combine two datasets 

# select columns what we need
athelet_df = athelet_df[['Name', 'CountryName', 'Year', 'Medal']]
gdp_df = gdp_df[['CountryName', 'Year', 'GDP']]


# Dropping rows where value is NA
athelet_df.dropna(subset=['Medal'], inplace=True)
gdp_df.dropna(subset=['GDP'], inplace=True)

# ## Merging datasets on 'Team' and 'Year'
merged_df = pd.merge(athelet_df, gdp_df, on=['CountryName', 'Year'])

# check the dataset
print(merged_df.shape)
print(merged_df.head())

(18307, 5)
                       Name CountryName  Year   Medal           GDP
0  Juhamatti Tapio Aaltonen     Finland  2014  Bronze  2.748628e+11
1         Aleksander Barkov     Finland  2014  Bronze  2.748628e+11
2    Mikael Antero Granlund     Finland  2014  Bronze  2.748628e+11
3            Juuso Hietanen     Finland  2014  Bronze  2.748628e+11
4            Jarkko Immonen     Finland  2014  Bronze  2.748628e+11


In [48]:
## Quantify Olympic performance

# Assign points to medals
medal_points = {'Gold': 3, 'Silver': 2, 'Bronze': 1}
merged_df['MedalPoints'] = merged_df['Medal'].map(medal_points)

# Group by Country and Year to calculate total points
performance_df = merged_df.groupby(['CountryName', 'Year']).agg(
    TotalMedals=('Medal', 'count'),  # Count of medals
    TotalPoints=('MedalPoints', 'sum')  # Sum of points
).reset_index()

# Merging the performance data with GDP data
final_df = pd.merge(performance_df, gdp_df, on=['CountryName', 'Year'])

# Display the final DataFrame
print(final_df.head())

# Sort the data
final_df = final_df.sort_values(by=['CountryName', 'Year'])
print(final_df.head())

   CountryName  Year  TotalMedals  TotalPoints           GDP
0  Afghanistan  2008            1            1  1.024977e+10
1  Afghanistan  2012            1            1  2.020357e+10
2      Algeria  1984            2            2  5.369855e+10
3      Algeria  1992            2            4  4.800313e+10
4      Algeria  1996            3            7  4.694155e+10
   CountryName  Year  TotalMedals  TotalPoints           GDP
0  Afghanistan  2008            1            1  1.024977e+10
1  Afghanistan  2012            1            1  2.020357e+10
2      Algeria  1984            2            2  5.369855e+10
3      Algeria  1992            2            4  4.800313e+10
4      Algeria  1996            3            7  4.694155e+10
